<!--BOOK_INFORMATION-->
<a href="https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv" target="_blank"><img align="left" src="data/cover.jpg" style="width: 76px; height: 100px; background: white; padding: 1px; border: 1px solid black; margin-right:10px;"></a>
*This notebook contains an excerpt from the upcoming book [Machine Learning for OpenCV](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv) by Michael Beyeler (expected Aug 2017).
The code is released under the [MIT license](https://opensource.org/licenses/MIT),
and is available on [GitHub](https://github.com/mbeyeler/opencv-machine-learning).*

*Note that this excerpt contains only the raw code - the book is rich with additional explanations and illustrations.
If you find this content useful, please consider supporting the work by
[buying the book](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv)!*

<!--NAVIGATION-->
< [None](10.00-Combining-Different-Algorithms-Into-an-Ensemble.ipynb) | [Contents](../README.md) | [None](12.00-Conclusion.ipynb) >

# Understanding Cross-Validation

## Manually implementing cross-validation in OpenCV

In [1]:
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split
X_fold1, X_fold2, y_fold1, y_fold2 = train_test_split(
    X, y, random_state=37, train_size=0.5
)

In [3]:
import cv2
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)

In [4]:
knn.train(X_fold1, cv2.ml.ROW_SAMPLE, y_fold1)
_, y_hat_fold2 = knn.predict(X_fold2)

In [5]:
knn.train(X_fold2, cv2.ml.ROW_SAMPLE, y_fold2)
_, y_hat_fold1 = knn.predict(X_fold1)

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y_fold1, y_hat_fold1)

0.92000000000000004

In [7]:
accuracy_score(y_fold2, y_hat_fold2)

0.88

## Automating cross-validation using scikit-learn

In [8]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [9]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
scores

array([ 0.96666667,  0.96666667,  0.93333333,  0.93333333,  1.        ])

In [10]:
scores.mean(), scores.std()

(0.95999999999999996, 0.024944382578492935)

## Implementing leave-one-out cross-validation

In [11]:
from sklearn.model_selection import LeaveOneOut

In [12]:
scores = cross_val_score(model, X, y, cv=LeaveOneOut())

In [13]:
scores

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [14]:
scores.mean(), scores.std()

(0.95999999999999996, 0.19595917942265423)

# Understanding bootstrapping

In [15]:
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)

In [16]:
idx_boot = np.random.choice(len(X), size=len(X), replace=True)
X_boot = X[idx_boot, :]
y_boot = y[idx_boot]

In [17]:
idx_oob = np.array([x not in idx_boot
                    for x in np.arange(len(X))], dtype=np.bool)
X_oob = X[idx_oob, :]
y_oob = y[idx_oob]

In [18]:
knn.train(X_boot, cv2.ml.ROW_SAMPLE, y_boot)

True

In [19]:
_, y_hat = knn.predict(X_oob)
accuracy_score(y_oob, y_hat)

0.98113207547169812

In [20]:
def yield_bootstrap(model, X, y, n_iter=10000):
    for _ in range(n_iter):
        # train the classifier on bootstrap
        idx_boot = np.random.choice(len(X), size=len(X),
                                    replace=True)
        X_boot = X[idx_boot, :]
        y_boot = y[idx_boot]
        knn.train(X_boot, cv2.ml.ROW_SAMPLE, y_boot)
        
        # test classifier on out-of-bag examples
        idx_oob = np.array([x not in idx_boot
                            for x in np.arange(len(X))],
                           dtype=np.bool)
        X_oob = X[idx_oob, :]
        y_oob = y[idx_oob]
        _, y_hat = knn.predict(X_oob)
        
        # return accuracy
        yield accuracy_score(y_oob, y_hat)

In [21]:
np.random.seed(42)

In [22]:
list(yield_bootstrap(knn, X, y, n_iter=10))

[0.98333333333333328,
 0.93650793650793651,
 0.92452830188679247,
 0.92307692307692313,
 0.94545454545454544,
 0.94736842105263153,
 0.98148148148148151,
 0.96078431372549022,
 0.93220338983050843,
 0.96610169491525422]

In [23]:
acc = list(yield_bootstrap(knn, X, y, n_iter=1000))
np.mean(acc), np.std(acc)

(0.95524155136419198, 0.022040380995646654)

In [24]:
acc = list(yield_bootstrap(knn, X, y, n_iter=10000))
np.mean(acc), np.std(acc)

(0.95501528733009422, 0.021778543317079499)

# Implementing Student's t-test

In [25]:
from scipy.stats import ttest_ind

In [26]:
scores_a = [1, 1, 1, 1, 1]
scores_b = [0, 0, 0, 0, 0]

In [27]:
ttest_ind(scores_a, scores_b)

Ttest_indResult(statistic=inf, pvalue=0.0)

In [28]:
scores_a = [0.9, 0.9, 0.9, 0.8, 0.8]
scores_b = [0.8, 0.8, 0.9, 0.9, 0.9]
ttest_ind(scores_a, scores_b)

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [29]:
k1 = KNeighborsClassifier(n_neighbors=1)
scores_k1 = cross_val_score(k1, X, y, cv=10)
np.mean(scores_k1), np.std(scores_k1)

(0.95999999999999996, 0.053333333333333323)

In [30]:
k3 = KNeighborsClassifier(n_neighbors=3)
scores_k3 = cross_val_score(k3, X, y, cv=10)
np.mean(scores_k3), np.std(scores_k3)

(0.96666666666666656, 0.044721359549995787)

In [31]:
ttest_ind(scores_k1, scores_k3)

Ttest_indResult(statistic=-0.2873478855663425, pvalue=0.77712784875052965)

## Implementing McNemar's test

In [32]:
from scipy.stats import binom
def mcnemar_midp(b, c):
    """
    Compute McNemar's test using the "mid-p" variant suggested by:
    
    M.W. Fagerland, S. Lydersen, P. Laake. 2013. The McNemar test for 
    binary matched-pairs data: Mid-p and asymptotic are better than exact 
    conditional. BMC Medical Research Methodology 13: 91.
    
    `b` is the number of observations correctly labeled by the first---but 
    not the second---system; `c` is the number of observations correctly 
    labeled by the second---but not the first---system.
    """
    n = b + c
    x = min(b, c)
    dist = binom(n, .5)
    p = 2. * dist.cdf(x)
    midp = p - dist.pmf(x)
    return midp

In [33]:
scores_a = np.array([1, 1, 1, 1, 1])
scores_b = np.array([0, 0, 0, 0, 0])

In [34]:
a1_b0 = scores_a * (1 - scores_b)
a1_b0

array([1, 1, 1, 1, 1])

In [35]:
a0_b1 = (1 - scores_a) * scores_b
a0_b1

array([0, 0, 0, 0, 0])

In [36]:
mcnemar_midp(a1_b0.sum(), a0_b1.sum())

0.03125

In [37]:
scores_k1 = cross_val_score(k1, X, y, cv=LeaveOneOut())
scores_k3 = cross_val_score(k3, X, y, cv=LeaveOneOut())

In [38]:
np.sum(scores_k1 * (1 - scores_k3))

0.0

In [39]:
np.sum((1 - scores_k3) * scores_k3)

0.0

In [40]:
mcnemar_midp(np.sum(scores_k1 * (1 - scores_k3)),
             np.sum((1 - scores_k1) * scores_k3))

1.0

<!--NAVIGATION-->
< [None](10.00-Combining-Different-Algorithms-Into-an-Ensemble.ipynb) | [Contents](../README.md) | [None](12.00-Conclusion.ipynb) >